# Module 2.A - Where Data Comes From (Real-World Pipeline Starter)

### Core dataset for the whole module 

**NYC 311 Service Requests (2020 - present)** (NYC Open Data/Socrata)  
Why this dataset works for learning:
* **Real mess:** missing values, inconsistent strings, free-text fields, and "weird" categories.
* **Real scale:** the full dataset is huge, so must learn how to pull a *slice*
* **Multiple access modes:** the same data is available as **CSV, API (JSON), SQL**
* **Real change over time:** fields and value distributions can shift (schema drift)

Will also create two supporting assets that will be reused later:
* **Data dictionary Excel file:** (`.xlsx`) from the publisher (documentation)
* **Borough reference table:** either scraped from the web or created as a seed file

### What will be built in 2.A  

Will produce a local, module-scoped workspace. Organized by module and stored as described:  
```bash
~/work/m2/data/
    raw/          # downloaded files, API responses
    reference/    # lookup tables, dictionaries
    warehouse/    # SQLite databases
```

Later notebooks will assume these exist:
* **2.B** Data quality: missingness, duplicates, inconsistent categories, schema drift
* **2.C** Wrangling: groupby, joins, string cleaning, feature construction
* **2.D** Scaling: incremental refresh, "raw &rarr; staged &rarr; curated" thinking
* **2.E** Outliers/validation: response times, anomaly checks, "is this plausible?" rules

The goal in 2.A is not perfect cleaning. It is learning how to acquire data reliably and keep the process reproducible

## Setup (requests, BeautifulSoup, and a writable workspace)

Common libraries:
* **requests:** a simple way to make HTTP requests
* **BeautifulSoup:** parses HTML to extract pieces

In [2]:
from __future__ import annotations

import json
import sqlite3
from datetime import datetime, timedelta, timezone
from pathlib import Path
from typing import Any, Dict, List

import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# Writable workspace (module-scoped)
WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR   = MODULE_DIR / "data"
RAW_DIR    = DATA_DIR / "raw"
REF_DIR    = DATA_DIR / "reference"
WH_DIR     = DATA_DIR / "warehouse"

for d in [RAW_DIR, REF_DIR, WH_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Writable Module 2 data workspace ready:")
print(" ", DATA_DIR.resolve())

Writable Module 2 data workspace ready:
  /home/glake/Nextcloud/Classwork/CS6678 - Advanced Machine Learning/Jupyter Notebooks/work/m2/data


## A.0 - Source Audit Template

Before cleaning, provide answers to:  
* What does one row represent
* What system produced it?
* What time range does it cover?
* What are known limitations?
* Which fields look risky (missing, free-text, inconsistent categories)?

We will keep a small structured dictionary of notes that can be reused later

In [3]:
source_audit = {
    "dataset_name": "NYC 311 Service Requests (2020-present)",
    "publisher": "NYC Open Data / 311",
    "where_it_comes_from": "City 311 request intake system (customer service requests routed to agencies).",
    "unit_of_analysis": "Each row represents one 311 service request.", 
    "time_grain": "Requests are created continuously; rows include timestamps for created/closed when available.",
    "known_limitations": [
        "Many fields are optional depending on request type (expect missingness).",
        "Free-text fields (descriptor/address) can be inconsistent and messy.",
        "The dataset is continuously updated; results can change between runs.",
    ],
    "Notes 1/30/26": []
}

source_audit

{'dataset_name': 'NYC 311 Service Requests (2020-present)',
 'publisher': 'NYC Open Data / 311',
 'where_it_comes_from': 'City 311 request intake system (customer service requests routed to agencies).',
 'unit_of_analysis': 'Each row represents one 311 service request.',
 'time_grain': 'Requests are created continuously; rows include timestamps for created/closed when available.',
 'known_limitations': ['Many fields are optional depending on request type (expect missingness).',
  'Free-text fields (descriptor/address) can be inconsistent and messy.',
  'The dataset is continuously updated; results can change between runs.'],
 'Notes 1/30/26': []}

## A.1 Files (CSV): Download a Reproducible Slice

Large public datasets are often too big to download in full for learning. A useful technique is to define a slice that is:  
* small enough to iterate quickly (seconds, not minutes)
* recent enough to include real mess
* refreshable

We will pull the **last 14 days** of NYC 311 requests as a CSV  

**Note on Socrata Timestamps**  

NYC Open Data uses Socrata. Many timestamp fields are "floating timestamps" and expect ISO8601 without timezone suffixes (No Z, No +00:00) in the query string. So we format timestamps like: `2026-01-04T04:03:21`

In [4]:
NYC311_BASE = "https://data.cityofnewyork.us/resource/erm2-nwe9"

# Stable subset of columns that will be reused across Module 2.
NYC311_COLUMNS = [
    "unique_key",
    "created_date",
    "closed_date",
    "agency",
    "agency_name",
    "complaint_type",
    "descriptor",
    "status",
    "borough",
    "incident_zip",
    "incident_address",
    "street_name",
    "city",
    "latitude",
    "longitude",
]

def iso_floating(dt: datetime) -> str:
    """
    Socrata floating timestamps expect ISO8601 without timezone suffix.
    We will drop tzinfo and milliseconds to be conservative
    """
    dt = dt.astimezone(timezone.utc).replace(tzinfo=None)
    return dt.strftime("%Y-%m-%dT%H:%M:%S")

def download_if_missing(url: str, path: Path, params: dict | None = None, timeout: int = 30) -> Path:
    """
    Download a URL to disk only if the file is not already cached.
    We chace downloads so later notebooks (2.B - 2.E) can reuse the same files
    without hammering the public API repeatedly.
    """
    if path.exists() and path.stat().st_size > 0:
        print("Used cached:", path)
        return path

    print("Downloading:", url)
    r = requests.get(url, params=params, timeout=timeout)

    if r.status_code >= 400:
        print("Status:", r.status_code)
        print("Body (first 300 chars):", r.text[:300])

    r.raise_for_status()
    path.write_bytes(r.content)
    print("Saved:", path, f"({path.stat().st_size/1e6:.2f} MB)")
    return path

def socrata_csv_params(days: int=14, limit: int=5000) -> dict:
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=days)

    select = ",".join(NYC311_COLUMNS)
    where = (
        f"created_date >= '{iso_floating(start)}'"
        f"AND created_date < '{iso_floating(end)}'"
    )
    return {"$select": select, "$where": where, "$order": "created_date DESC", "$limit": limit}

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
params = socrata_csv_params(days=14, limit=50000)

print("Where clause:", params["$where"])
download_if_missing(f"{NYC311_BASE}.csv", CSV_PATH, params=params)

Where clause: created_date >= '2026-01-17T14:18:35'AND created_date < '2026-01-31T14:18:35'
Used cached: work/m2/data/raw/nyc311_last14d.csv


PosixPath('work/m2/data/raw/nyc311_last14d.csv')

### Load the CSV and do a quick source audit

In [5]:
df_csv = pd.read_csv(CSV_PATH)
df_csv.head(5)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [6]:
df_csv.dtypes

unique_key            int64
created_date         object
closed_date          object
agency               object
agency_name          object
complaint_type       object
descriptor           object
status               object
borough              object
incident_zip        float64
incident_address     object
street_name          object
city                 object
latitude            float64
longitude           float64
dtype: object

In [7]:
df_csv.isna().mean().sort_values(ascending=False).head(10)

closed_date         0.60766
city                0.07640
street_name         0.02592
incident_address    0.02588
latitude            0.01222
longitude           0.01222
incident_zip        0.00732
descriptor          0.00680
complaint_type      0.00000
agency              0.00000
dtype: float64

**Update Source Audit**  

Add 3 observations to `source_audit["today_notes"]`

In [11]:
# clear current notes so that the cell does not create duplicate entries
source_audit["Notes 1/30/26"] = []
source_audit["Notes 1/30/26"].append("incident_zip does not need to be a float64")
source_audit["Notes 1/30/26"].append("There is at least one entry that is incomplete")
source_audit["Notes 1/30/26"].append("closed_date is often missing")

source_audit

{'dataset_name': 'NYC 311 Service Requests (2020-present)',
 'publisher': 'NYC Open Data / 311',
 'where_it_comes_from': 'City 311 request intake system (customer service requests routed to agencies).',
 'unit_of_analysis': 'Each row represents one 311 service request.',
 'time_grain': 'Requests are created continuously; rows include timestamps for created/closed when available.',
 'known_limitations': ['Many fields are optional depending on request type (expect missingness).',
  'Free-text fields (descriptor/address) can be inconsistent and messy.',
  'The dataset is continuously updated; results can change between runs.'],
 'Notes 1/30/26': ['incident_zip does not need to be a float64',
  'There is at least one entry that is incomplete',
  'closed_date is often missing']}

## A.2 SQL databases (SQLite): Land Raw Data Into a Local Warehouse

In [12]:
DB_PATH = WH_DIR / "module2.db"
conn = sqlite3.connect(DB_PATH)
print("DB:", DB_PATH.resolve())

DB: /home/glake/Nextcloud/Classwork/CS6678 - Advanced Machine Learning/Jupyter Notebooks/work/m2/data/warehouse/module2.db


### Write the raw CSV into a table

In [13]:
df_csv.to_sql("nyc311_raw", conn, if_exists="replace", index=False)

cur = conn.cursor()
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_created_date ON nyc311_raw(created_date)")
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_borough ON nyc311_raw(borough)")
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_complaint_type ON nyc311_raw(complaint_type)")
conn.commit()

print("Rows in nyc311_raw:", cur.execute("SELECT COUNT(*) FROM nyc311_raw").fetchone()[0])

Rows in nyc311_raw: 50000


### SQL sanity checks

In [14]:
q = """
SELECT borough, COUNT(*) AS n
FROM nyc311_raw
GROUP BY borough
ORDER BY n DESC
"""

pd.read_sql_query(q, conn).head(20)

,borough,n
0,BROOKLYN,15654
1,QUEENS,11831
2,BRONX,10806
3,MANHATTAN,8876
4,STATEN ISLAND,2798
5,Unspecified,35


In [15]:
q = """
SELECT complaint_type, COUNT(*) AS n
FROM nyc311_raw
GROUP BY complaint_type
ORDER BY n DESC
LIMIT 15
"""

pd.read_sql_query(q, conn)

,complaint_type,n
0,Snow or Ice,12376
1,HEAT/HOT WATER,11890
2,Illegal Parking,4922
3,Blocked Driveway,2790
4,Noise - Residential,2492
5,UNSANITARY CONDITION,1288
6,PLUMBING,1212
7,PAINT/PLASTER,819
8,DOOR/WINDOW,773
9,WATER LEAK,750


## A.3 APIs: query the same dataset via Socrata (JSON)

In [16]:
def socrata_json(days: int=14, limit: int=1000, offset: int=0) -> List[Dict[str, Any]]:
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=days)

    select = ",".join(NYC311_COLUMNS)
    where = (
        f"created_date >= '{iso_floating(start)}'"
        f"AND created_date < '{iso_floating(end)}'"
    )

    params = {"$select": select, "$where": where, "$order": "created_date DESC", "$limit": limit, "$offset": offset}
    r = requests.get(f"{NYC311_BASE}.json", params=params, timeout=30)

    if r.status_code >= 400:
        print("Status:", r.status_code)
        print("Where:", where)
        print("Body (first 300 chars):", r.text[300])
    r.raise_for_status()
    return r.json()

page1 = socrata_json(days=14, limit=1000, offset=0)
page2 = socrata_json(days=14, limit=1000, offset=1000)
records = page1 + page2

print("Records fetched (JSON):", len(records))
print("Example record keys:", list(records[0].keys())[:12])

Records fetched (JSON): 2000
Example record keys: ['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'status', 'borough']


### Save raw JSON

In [17]:
JSON_PATH = RAW_DIR / "nyc311_last14d.json"
if not JSON_PATH.exists():
    JSON_PATH.write_text(json.dumps(records, indent=2))
    print("Saved:", JSON_PATH)
else:
    print("Using cached:", JSON_PATH)

df_api = pd.json_normalize(records)
df_api.head(3)

Saved: work/m2/data/raw/nyc311_last14d.json


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude,closed_date
0,67720523,2026-01-30T01:51:21.000,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.64978696357874,-73.95854980692795,NaN
2,67758820,2026-01-30T01:50:53.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.80049840941944,-73.96567975561733,NaN


### Compare CSV vs API quickly

In [18]:
comparison_notes = {
    "csv_columns": len(df_csv.columns),
    "api_columns": len(df_api.columns),
    "columns_only_in_csv": sorted(set(df_csv.columns) - set(df_api.columns))[:20],
    "columns_only_in_api": sorted(set(df_api.columns) - set(df_csv.columns))[:20],
    "csv_missing_rate_top10": df_csv.isna().mean().sort_values(ascending=False).head(10).to_dict(),
    "api_missing_rate_top10": df_api.isna().mean().sort_values(ascending=False).head(10).to_dict(),
}

comparison_notes


{'csv_columns': 15,
 'api_columns': 15,
 'columns_only_in_csv': [],
 'columns_only_in_api': [],
 'csv_missing_rate_top10': {'closed_date': 0.60766,
  'city': 0.0764,
  'street_name': 0.02592,
  'incident_address': 0.02588,
  'latitude': 0.01222,
  'longitude': 0.01222,
  'incident_zip': 0.00732,
  'descriptor': 0.0068,
  'complaint_type': 0.0,
  'agency': 0.0},
 'api_missing_rate_top10': {'closed_date': 0.649,
  'city': 0.046,
  'street_name': 0.024,
  'incident_address': 0.024,
  'latitude': 0.019,
  'longitude': 0.019,
  'incident_zip': 0.0075,
  'status': 0.0,
  'descriptor': 0.0,
  'complaint_type': 0.0}}

## A.4 Reference Data: Borough Lookup (Scrape OR Seed File)

Try **Scrape** first, if it fails (403), fall back to **Seed File**

In [19]:
BORO_PATH = REF_DIR / "nyc_boroughs.csv"

if BORO_PATH.exists():
    print("Using cached:", BORO_PATH)
    df_boro = pd.read_csv(BORO_PATH)
    display(df_boro)
else:
    WIKI_URL = "https://en.wikipedia.org/wiki/Boroughs_of_New_York_City"
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (educational; JupyterLab) requests",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
    }

    try:
        r = requests.get(WIKI_URL, headers=HEADERS, timeout=30)
        if r.status_code >= 400:
            raise RuntimeError(f"HTTP {r.status_code} from Wikipedia")

        soup = BeautifulSoup(r.text, "html.parser")

        tables = soup.find_all("table", class_="wikitable")
        target = None
        for t in tables:
            header_text = t.get_text(" ", strip=True).lower()
            if "borough" in header_text and "population" in header_text:
                target = t
                break

        if target is None:
            raise RuntimeError("Could not find expected borough table (page structure may have changed).")

        rows = []
        for tr in target.find_all("tr")[1:]:
            tds = tr.find_all(["th", "td"])
            if not tds:
                continue
            borough = tds[0].get_text(" ", strip=True).strip()
            if borough and borough.lower() != "borough":
                rows.append({"borough": borough})

        df_boro = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)
        df_boro.to_csv(BORO_PATH, index=False)
        print("Option A success: scraped boroughs and saved:", BORO_PATH)
    
    except Exception as e:
        print("Option A failed (scraping). Reason:", e)
        print("Falling back to Option B (seed file).")
    
        df_boro = pd.DataFrame([
            {"borough": "BRONX"},
            {"borough": "BROOKLYN"},
            {"borough": "MANHATTAN"},
            {"borough": "QUEENS"},
            {"borough": "STATEN ISLAND"},
        ])
        df_boro.to_csv(BORO_PATH, index=False)
        print("Option B success: saved seed borough table:", BORO_PATH)
    
    display(df_boro)

Option A success: scraped boroughs and saved: work/m2/data/reference/nyc_boroughs.csv


,borough
0,Jurisdiction
1,The Bronx
2,Brooklyn
3,Manhattan
4,Queens
5,Staten Island
6,City of New York
7,State of New York
8,Sources : [ 3 ] [ 4 ] [ 5 ] [ 6 ] and see indi...


**Notes:**  
* Scraping can break even if the code is correct (403, HTML changes)
* In many teams, seed files are the standard approach for small reference tables.

## A.5 Excel: Download the 311 data dictionary (XLSX)

In [20]:
DATA_DICTIONARY_URL = (
    "https://nycopendata.socrata.com/api/views/erm2-nwe9/files/"
    "b372b884-f86a-453b-ba16-1fe06ce9d212?download=true&filename=311_ServiceRequest_2010-Present_DataDictionary_Updated_2023.xlsx"
)

XLSX_PATH = REF_DIR / "nyc311_data_dictionary.xlsx"
download_if_missing(DATA_DICTIONARY_URL, XLSX_PATH)

Downloading: https://nycopendata.socrata.com/api/views/erm2-nwe9/files/b372b884-f86a-453b-ba16-1fe06ce9d212?download=true&filename=311_ServiceRequest_2010-Present_DataDictionary_Updated_2023.xlsx
Saved: work/m2/data/reference/nyc311_data_dictionary.xlsx (0.50 MB)


PosixPath('work/m2/data/reference/nyc311_data_dictionary.xlsx')

In [22]:
import openpyxl
wb = openpyxl.load_workbook(XLSX_PATH, read_only=True)
wb.sheetnames

['Dataset Information',
 'Column Information',
 'All Agencies Complaint<>Details',
 'HPD Complaint<>Details',
 'Dataset Revision History',
 'Primer Page & InternaI Informat',
 'Hidden_Frequencies',
 'Hidden_Agencies',
 'Hidden_DataTypes']

In [23]:
sheet = wb.sheetnames[0]
df_dict_preview = pd.read_excel(XLSX_PATH, sheet_name=sheet)
df_dict_preview.head(20)

/home/glake/miniconda3/envs/ml/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Unnamed: 0,Unnamed: 1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,Data Dictionary - Dataset Information,NaN
7,Dataset Name,311 Service Requests from 2010 to Present
8,Dataset URL,https://data.cityofnewyork.us/Social-Services/...
9,Data Provided by\nThe name of the NYC agency p...,311


Many data dictionaries are human-formatted spreadsheets with title blocks and notes. The first read may look messy (Unnamed columns, NaNs). That is normal

## A.6 Wrap-up: Verify Reusable Artifacts Exist

In [24]:
paths = [CSV_PATH, JSON_PATH, BORO_PATH, XLSX_PATH, DB_PATH]
[(str(p), p.exists(), round(p.stat().st_size/1e6, 3) if p.exists() else None) for p in paths]

[('work/m2/data/raw/nyc311_last14d.csv', True, 12.143),
 ('work/m2/data/raw/nyc311_last14d.json', True, 1.036),
 ('work/m2/data/reference/nyc_boroughs.csv', True, 0.0),
 ('work/m2/data/reference/nyc311_data_dictionary.xlsx', True, 0.497),
 ('work/m2/data/warehouse/module2.db', True, 13.627)]

In [25]:
reflection = {
    "row_definition": "Each row represents one 311 service request.",
    "incremental_key": "",
    "high_risk_columns": [],
    "sensitive_columns": [],
}
reflection


{'row_definition': 'Each row represents one 311 service request.',
 'incremental_key': '',
 'high_risk_columns': [],
 'sensitive_columns': []}

# Module 2.B - Data Quality & Structure (NYC 311)

in this notebook we will practice a realistic workflow.  
1. **Load** the same NYC311 slice created in 2.A
2. **Audit quality:** missingness, duplicates, inconsistencies
3. **Detect drift:** "today vs yesterday" schema differences
4. Produce a short **Data Quality Report** that will be reused in later modules

**Note**:  
We are not trying to make the dataset perfect.  
We are trying to make it **trustworthy enough for a special decision** and to document what was done.

## 2B.0 - Setup

This notebook expects that **Module 2.A** has already been ran and that a module data workspace has been created  

`/work/m2/data/`

In [7]:
from __future__ import annotations

from datetime import datetime, timedelta, timezone
from pathlib import Path
import json
import re

import numpy as np
import pandas as pd
import requests

pd.set_option("display.max_columns", 80)
pd.set_option("display.width", 140)

WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR = MODULE_DIR / "data"

RAW_DIR = DATA_DIR / "raw"
REF_DIR = DATA_DIR / "reference"
WH_DIR  = DATA_DIR / "warehouse"

for d in [RAW_DIR, REF_DIR, WH_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
JSON_PATH = RAW_DIR / "nyc311_last14d.json"  # optional if you created via API in 2.A

print("Data directory:", DATA_DIR.resolve())
print("CSV exists?", CSV_PATH.exists(), "|", CSV_PATH)
print("JSON exists?", JSON_PATH.exists(), "|", JSON_PATH)

Data directory: /home/glake/Nextcloud/Classwork/CS6678 - Advanced Machine Learning/Jupyter Notebooks/work/m2/data
CSV exists? True | work/m2/data/raw/nyc311_last14d.csv
JSON exists? True | work/m2/data/raw/nyc311_last14d.json


## 2B.1 - Load the Data and Establish a Row Definition

**Row definition (target):** Each row represents one 311 service request, identified by `unique_key`  

Before cleaning, we load as-is and profile the mess

In [8]:
df_raw = pd.read_csv(CSV_PATH)
print("Shape:", df_raw.shape)
df_raw.head(5)

Shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [9]:
df_raw.dtypes.to_frame("dtype").T

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
dtype,int64,object,object,object,object,object,object,object,object,float64,object,object,object,float64,float64


### Create quality notes  

Write row definition and first impressions. Add 3-6 bullets:
* What does one row represent?
* What columns look risky?
* What looks like "encoded missingness"?

In [28]:
quality_notes = {
    "row_definition": "Each row is one 311 service request (unique_key) created at created_date.",
    "data_type_notes": [
        "incident_zip should be treated as a string to preserve leading zeros.",
        "descriptor/incident_address are free-text and likely inconsistent.",
    ],
    "completeness_issues": [],
    "date_added": "1/30/26"
}
quality_notes

{'row_definition': 'Each row is one 311 service request (unique_key) created at created_date.',
 'data_type_notes': ['incident_zip should be treated as a string to preserve leading zeros.',
  'descriptor/incident_address are free-text and likely inconsistent.'],
 'completeness_issues': [],
 'date_added': '1/30/26'}

## 2B.2 - Missing data: Measure, then explain

First rule: **do not impute yet**  
Start by quantifying missingness and asking why it might be missing

In [10]:
missing_rate = df_raw.isna().mean().sort_values(ascending=False)
missing_rate.head(15)

closed_date         0.60766
city                0.07640
street_name         0.02592
incident_address    0.02588
latitude            0.01222
longitude           0.01222
incident_zip        0.00732
descriptor          0.00680
complaint_type      0.00000
agency              0.00000
created_date        0.00000
unique_key          0.00000
agency_name         0.00000
status              0.00000
borough             0.00000
dtype: float64

### Normalize common "missing" encodings  

Many real datasets use sentinel values like `""`, `"UNKNOWN"`, `"N/A"`, or whitespace  

Create a `df` as a cleaned copy while keeping `df_raw` unchanged.

In [11]:
df = df_raw.copy()

MISSING_TOKENS = {"", " ", "  ", "UNKNOWN", "Unknown", "N/A", "NA", "NULL", "null"}

for col in df.columns:
    if df[col].dtype == "object":
        # Normalize whitespace-only strings
        df[col] = df[col].astype(str).str.strip()
        df.loc[df[col].isin(MISSING_TOKENS), col] = np.nan

# Parse timestamps (coerce errors to NaT)
for col in ["created_date", "closed_date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Treat ZIP as string (preserve leading zeros, handle NaN)
if "incident_zip" in df.columns:
    df["incident_zip"] = df["incident_zip"].astype("string")

df.isna().mean().sort_values(ascending=False).head(15)

closed_date         0.60766
longitude           0.01222
latitude            0.01222
incident_zip        0.00732
agency              0.00000
unique_key          0.00000
created_date        0.00000
descriptor          0.00000
complaint_type      0.00000
agency_name         0.00000
status              0.00000
incident_address    0.00000
borough             0.00000
city                0.00000
street_name         0.00000
dtype: float64

### Missingness by group  

If missingness varies a lot by group (borough, agency, complaint type), it is often not random, and that affects modeling and fairness  

In [12]:
# Choose a column with substantial missingness to inspect
target_col = "incident_address" if "incident_address" in df.columns else df.columns[df.isna().mean().idxmax()]
print("Inspecting missingness for:", target_col)

group_col = "borough" if "borough" in df.columns else "agency"
tmp = df[[group_col, target_col]].copy()
tmp["is_missing"] = tmp[target_col].isna().astype(int)

missing_by_group = tmp.groupby(group_col)["is_missing"].mean().sort_values(ascending=False)
missing_by_group.head(10)

Inspecting missingness for: incident_address


borough
BRONX            0.0
BROOKLYN         0.0
MANHATTAN        0.0
QUEENS           0.0
STATEN ISLAND    0.0
Unspecified      0.0
Name: is_missing, dtype: float64

In [13]:
# Check another column - Closed Date
target_col = "closed_date" if "closed_date" in df.columns else df.columns[df.isna().mean().idxmax()]
print("Closed Date missingness for:", target_col)

group_col = "borough" if "borough" in df.columns else "agency"
tmp = df[[group_col, target_col]].copy()
tmp["is_missing"] = tmp[target_col].isna().astype(int)

missing_by_group = tmp.groupby(group_col)["is_missing"].mean().sort_values(ascending=False)
missing_by_group.head(10)


Closed Date missingness for: closed_date


borough
Unspecified      0.857143
STATEN ISLAND    0.728735
MANHATTAN        0.631478
BROOKLYN         0.598952
BRONX            0.593374
QUEENS           0.584989
Name: is_missing, dtype: float64

### Task - Missingness Hypothesis  

Pick one column with missingness > 20% and answer:
1. What might cause it to be missing?
2. Is it "optional by design" (depends on complaint type)?
3. Would dropping rows with missing values change what the data represents?

In [14]:
missingness_hypothesis = {
    "Column: closed_date" : "often missing because requests are still open, or closure not required",
    "More data" : "More data"
}

missingness_hypothesis

{'Column: closed_date': 'often missing because requests are still open, or closure not required',
 'More data': 'More data'}

## 2B.3 - Duplicates: Exact vs Entity Duplicates

For this dataset, `unique_key` should identify a request.  
* **Exact duplicates:** identical rows repeated (export/pipeline issues)
* **Key duplicates:** `unique_key` repeats (should be rare; indicates ingestion error)

In [15]:
# Exact duplicates across all columns
exact_dup_count = df.duplicated().sum()
print("Exact duplicate rows:", int(exact_dup_count))

# Duplicates in the request identifier
if "unique_key" in df.columns:
    key_dup_count = df["unique_key"].duplicated().sum()
    print("Duplicate unique_key values:", int(key_dup_count))
else:
    print("No unique_key column found.")

Exact duplicate rows: 0
Duplicate unique_key values: 0


In [16]:
if "unique_key" in df.columns and df["unique_key"].duplicated().any():
    dup_keys = df.loc[df["unique_key"].duplicated(keep=False), "unique_key"].head(10).tolist()
    df[df["unique_key"].isin(dup_keys)].sort_values("unique_key").head(20)
else:
    print("No key duplicates found (good).")

No key duplicates found (good).


### Decide a depupe rule (if needed)

If key duplicates exists, must choose a rule:
* keep the newest record (by created_date)
* keep the most complete record
* keep the first seen (not recommended unless you know ordering is stable)

In [17]:
def dedupe_by_key_keep_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    """Example rule: keep the row with the fewest missing values per key."""
    if key not in df_in.columns:
        return df_in.copy()

    df_tmp = df_in.copy()
    miss = df_tmp.isna().sum(axis=1)
    df_tmp["_missing_count"] = miss
    df_tmp = df_tmp.sort_values([key, "_missing_count"]).drop_duplicates(subset=[key], keep="first")
    return df_tmp.drop(columns=["_missing_count"])

df_deduped = dedupe_by_key_keep_most_complete(df, "unique_key")
print("Before:", df.shape, "| After:", df_deduped.shape)

Before: (50000, 15) | After: (50000, 15)


**Dedupe Notes:**  

If duplicates are present, document the rule. If none are present document what was checked.

In [18]:
dedupe_notes = {
    "exact_duplicates": "None found (or dropped them).",
    "unique_key_duplicates": "None found (or deduped by most complete row).",
    "rationale": "unique_key should be the request identifier; duplicates would indicate pipeline/export issues.",
    "deduplication_date": "1/30/26"
}
dedupe_notes

{'exact_duplicates': 'None found (or dropped them).',
 'unique_key_duplicates': 'None found (or deduped by most complete row).',
 'rationale': 'unique_key should be the request identifier; duplicates would indicate pipeline/export issues.',
 'deduplication_date': '1/30/26'}

## 2B.4 - Inconsistencies: Categories, Strings, and "Almost the Same" Values  

In this dataset, the biggest inconsistency risk tends to be  
* **casing / whitespace** (e.g. borough values)
* **free-text fields** (descriptor, address)
* **categorical sprawl** (complaint_type has many levels)

We will do light normalization and build a category audit

In [19]:
def value_counts_audit(df_in: pd.DataFrame, col: str, n: int = 15) -> pd.DataFrame:
    vc = df_in[col].astype("string").value_counts(dropna=False).head(n)
    out = vc.to_frame("count")
    out["share"] = out["count"] / len(df_in)
    return out

for col in ["borough", "agency", "status", "complaint_type"]:
    if col in df.columns:
        print("\n===", col, "===")
        display(value_counts_audit(df, col, n=12))


=== borough ===


,count,share
borough,,
BROOKLYN,15654,0.31308
QUEENS,11831,0.23662
BRONX,10806,0.21612
MANHATTAN,8876,0.17752
STATEN ISLAND,2798,0.05596
Unspecified,35,0.0007



=== agency ===


,count,share
agency,,
HPD,18618,0.37236
DSNY,13160,0.2632
NYPD,11651,0.23302
DOT,1980,0.0396
DEP,1339,0.02678
DOB,766,0.01532
DOHMH,624,0.01248
DHS,613,0.01226
DPR,531,0.01062



=== status ===


,count,share
status,,
Closed,18894,0.37788
Open,15728,0.31456
In Progress,15014,0.30028
Assigned,203,0.00406
Started,137,0.00274
Pending,22,0.00044
Unspecified,2,0.00004



=== complaint_type ===


,count,share
complaint_type,,
Snow or Ice,12376,0.24752
HEAT/HOT WATER,11890,0.2378
Illegal Parking,4922,0.09844
Blocked Driveway,2790,0.0558
Noise - Residential,2492,0.04984
UNSANITARY CONDITION,1288,0.02576
PLUMBING,1212,0.02424
PAINT/PLASTER,819,0.01638
DOOR/WINDOW,773,0.01546


### Normalize a few high-impact fields  

Do not destroy meaning - just remove noise

In [21]:
df2 = df_deduped.copy()

# Standardize borough to a small canonical set where possible
if "borough" in df2.columns:
    df2["borough"] = df2["borough"].astype("string").str.strip().str.upper()

    # Some datasets use "Unspecified" or blanks
    df2.loc[df2["borough"].isin(["", "NAN", "UNSPECIFIED"]), "borough"] = pd.NA

# Standardize status
if "status" in df2.columns:
    df2["status"] = df2["status"].astype("string").str.strip().str.title()

print("After normalization:")
if "borough" in df2.columns:
    display(value_counts_audit(df2, "borough", n=10))
if "status" in df2.columns:
    display(value_counts_audit(df2, "status", n=10))

After normalization:


,count,share
borough,,
BROOKLYN,15654,0.31308
QUEENS,11831,0.23662
BRONX,10806,0.21612
MANHATTAN,8876,0.17752
STATEN ISLAND,2798,0.05596
<NA>,35,0.0007


,count,share
status,,
Closed,18894,0.37788
Open,15728,0.31456
In Progress,15014,0.30028
Assigned,203,0.00406
Started,137,0.00274
Pending,22,0.00044
Unspecified,2,0.00004


In [22]:
category_policy_notes = {
    "status_column": {
        "valid_set": "Appears to be {Open, Closed, Assigned, Pending} (verify)",
        "cleaning_policy": "Strip whitespace, title-case, map near-duplicates (e.g., 'CLOSED' -> 'Closed').",
        "rare_values": "If rare statuses exist (<0.5%), consider grouping as 'Other' depending on analysis goal."
    }
}
category_policy_notes

{'status_column': {'valid_set': 'Appears to be {Open, Closed, Assigned, Pending} (verify)',
  'cleaning_policy': "Strip whitespace, title-case, map near-duplicates (e.g., 'CLOSED' -> 'Closed').",
  'rare_values': "If rare statuses exist (<0.5%), consider grouping as 'Other' depending on analysis goal."}}

## 2B.5 - Schema Drift: Compare Versions and Fail Safely

Drift can show up as:
* column added/removed
* type changes (number - string)
* semantic changes (same name, different meaning)

We will do two comparisons.  
1. **CSV vs JSON**
2. **yesterday vs today simulation**

In [23]:
def schema_signature(df_in: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({"column": df_in.columns, "dtype": [str(t) for t in df_in.dtypes]}).sort_values("column")

def schema_diff(sig_a: pd.DataFrame, sig_b: pd.DataFrame) -> dict:
    a_cols = set(sig_a["column"])
    b_cols = set(sig_b["column"])
    added = sorted(list(b_cols - a_cols))
    removed = sorted(list(a_cols - b_cols))

    merged = sig_a.merge(sig_b, on="column", how="outer", suffixes=("_a", "_b"))
    changed = merged[(merged["dtype_a"].notna()) & (merged["dtype_b"].notna()) & (merged["dtype_a"] != merged["dtype_b"])]
    return {
        "added": added,
        "removed": removed,
        "dtype_changed": changed.sort_values("column")
    }

sig_csv = schema_signature(df2)
sig_csv.head()

,column,dtype
3,agency,object
4,agency_name,object
8,borough,string
12,city,object
2,closed_date,datetime64[ns]


In [24]:
if JSON_PATH.exists():
    records = json.loads(JSON_PATH.read_text())
    df_json = pd.json_normalize(records)

    sig_json = schema_signature(df_json)
    diff = schema_diff(sig_csv, sig_json)

    print("=== CSV vs JSON schema drift ===")
    print("Added in JSON (vs CSV):", diff["added"][:20], "..." if len(diff["added"])>20 else "")
    print("Removed in JSON (vs CSV):", diff["removed"][:20], "..." if len(diff["removed"])>20 else "")
    print("\nDtype changes (top 20):")
    display(diff["dtype_changed"].head(20))
else:
    print("JSON not found. (That's okay.) Run 2.A API section to generate:", JSON_PATH)

=== CSV vs JSON schema drift ===
Added in JSON (vs CSV): [] 
Removed in JSON (vs CSV): [] 

Dtype changes (top 20):


,column,dtype_a,dtype_b
2,borough,string,object
4,closed_date,datetime64[ns],object
6,created_date,datetime64[ns],object
9,incident_zip,string,object
10,latitude,float64,object
11,longitude,float64,object
12,status,string,object
14,unique_key,int64,object


### Drift Simulation - intentially mutate a new version  

This is what drift feels like operationally

In [25]:
df_new = df2.copy()

# Simulate drift: rename a column and change a type
if "incident_zip" in df_new.columns:
    df_new = df_new.rename(columns={"incident_zip": "incident_zipcode"})  # column rename

if "unique_key" in df_new.columns:
    df_new["unique_key"] = df_new["unique_key"].astype(str)  # type change (int -> str)

sig_new = schema_signature(df_new)
diff2 = schema_diff(sig_csv, sig_new)

print("=== Today vs 'new version' ===")
print("Added:", diff2["added"])
print("Removed:", diff2["removed"])
print("\nDtype changes:")
display(diff2["dtype_changed"])

=== Today vs 'new version' ===
Added: ['incident_zipcode']
Removed: ['incident_zip']

Dtype changes:


,column,dtype_a,dtype_b
15,unique_key,int64,object


In [26]:
drift_strategy_notes = {
    "required_columns": ["unique_key", "created_date", "complaint_type", "status"],
    "policies": {
        "missing_required": "Fail hard and alert (stop pipeline).",
        "missing_optional": "Add column filled with nulls + log warning.",
        "dtype_changes": "Attempt safe coercion; if coercion fails above threshold, fail hard."
    }
}
drift_strategy_notes

{'required_columns': ['unique_key',
  'created_date',
  'complaint_type',
  'status'],
 'policies': {'missing_required': 'Fail hard and alert (stop pipeline).',
  'missing_optional': 'Add column filled with nulls + log warning.',
  'dtype_changes': 'Attempt safe coercion; if coercion fails above threshold, fail hard.'}}

## 2B.6 - Deliverable: Data Quality Report (draft)

This is a short artifact that can be reused in future modules  

**Include:**
1. Row definition
2. Biggest missingness findings + hypothesis
3. Dedupe checks + rule
4. Category policy note
5. Drift Strategy

In [29]:
report = {
    "dataset": "NYC 311 Service Requests (last 14 days slice)",
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "row_definition": "Each row represents one 311 service request (unique_key).",
    "quality_notes": quality_notes,
    "missingness_hypothesis": missingness_hypothesis,
    "dedupe_notes": dedupe_notes,
    "category_policy_notes": category_policy_notes,
    "drift_strategy_notes": drift_strategy_notes,
}

print(json.dumps(report, indent=2)[:2000] + "\n... (truncated preview) ...")

{
  "dataset": "NYC 311 Service Requests (last 14 days slice)",
  "generated_at_utc": "2026-02-01T17:16:54.060563+00:00",
  "row_definition": "Each row represents one 311 service request (unique_key).",
  "quality_notes": {
    "row_definition": "Each row is one 311 service request (unique_key) created at created_date.",
    "data_type_notes": [
      "incident_zip should be treated as a string to preserve leading zeros.",
      "descriptor/incident_address are free-text and likely inconsistent."
    ],
    "completeness_issues": [],
    "date_added": "1/30/26"
  },
  "missingness_hypothesis": {
    "Column: closed_date": "often missing because requests are still open, or closure not required",
    "More data": "More data"
  },
  "dedupe_notes": {
    "exact_duplicates": "None found (or dropped them).",
    "unique_key_duplicates": "None found (or deduped by most complete row).",
    "rationale": "unique_key should be the request identifier; duplicates would indicate pipeline/export is

### Save Report  

In [30]:
REPORT_PATH = REF_DIR / "quality_report.json"
REPORT_PATH.write_text(json.dumps(report, indent=2))
print("Saved report:", REPORT_PATH)

Saved report: work/m2/data/reference/quality_report.json


# Module 2.C - Data Wrangling and Transformation (NYC 311)

In this module, we will:  
1. Load the NYC 311 slice created in **2.A**
2. Reuse the quality assumptions from **2.B** (`quality_report.json`)
3. Perform common wrangling patterns in pandas:
    * selecting/filtering, creating columns
    * `groupby` + aggregation
    * merges/joins (with guardrails)
    * string cleanup + regex extraction
4. Construct a baseline-friendly **feature table** and save it for later modules.

## 2C.0 - Setup and Paths

This notebook assumes the module workspace:  

`/work/m2/data`

* Raw data from **2.A**: `raw/nyc311_last14d.csv`
* Quality report from **2.B**: `reference/quality_report.json`

We will write outputs to:

* `warehouse/nyc311_requests_features.parquet`
* `warehouse/nyc311_daily_features.parquet`


In [1]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timezone
import json

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)

WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR = MODULE_DIR / "data"

RAW_DIR = DATA_DIR / "raw"
REF_DIR = DATA_DIR / "reference"
WH_DIR  = DATA_DIR / "warehouse"

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
REPORT_PATH = REF_DIR / "quality_report.json"

OUT_REQUESTS = WH_DIR / "nyc311_requests_features.parquet"
OUT_DAILY    = WH_DIR / "nyc311_daily_features.parquet"

print("CSV:", CSV_PATH, "| exists?", CSV_PATH.exists())
print("Report:", REPORT_PATH, "| exists?", REPORT_PATH.exists())
print("Warehouse:", WH_DIR)

CSV: work/m2/data/raw/nyc311_last14d.csv | exists? True
Report: work/m2/data/reference/quality_report.json | exists? True
Warehouse: work/m2/data/warehouse


### Load the quality report from from 2.B

In [2]:
report = None
if REPORT_PATH.exists():
    report = json.loads(REPORT_PATH.read_text())
    print("Loaded report generated_at_utc:", report.get("generated_at_utc"))
    print("Row definition:", report.get("row_definition"))
else:
    print("No quality_report.json found. Run 2.B to generate it.")

# Optional: inspect notes
if report:
    print("\nTop-level keys:", sorted(report.keys()))

Loaded report generated_at_utc: 2026-02-01T17:16:54.060563+00:00
Row definition: Each row represents one 311 service request (unique_key).

Top-level keys: ['category_policy_notes', 'dataset', 'dedupe_notes', 'drift_strategy_notes', 'generated_at_utc', 'missingness_hypothesis', 'quality_notes', 'row_definition']


## 2C.1 - Load Data and Re-apply Light Normalization

We keep this normalization conservative and explainable.  

**Row definition:** Each row represents one 311 service request, identified by `unique_key`.

In [3]:
df_raw = pd.read_csv(CSV_PATH)
print("Raw shape:", df_raw.shape)
df_raw.head()

Raw shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [4]:
# Conservative normalization helpers (similar spirit 2.B)
def normalize_missing_strings(df: pd.DataFrame, missing_tokens: str[str]) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == "object":
            s = out[c].astype(str).str.strip()
            # Replace known tokens with NaN (exact match)
            s = s.replace({t: np.nan for t in missing_tokens})
            out[c] = s
    return out

MISSING_TOKENS = {"", " ", "  ", "UNKNOWN", "Unkown", "N/A", "NA", "NULL", "null"}

df = normalize_missing_strings(df_raw, MISSING_TOKENS)

# Parse timestamps
for c in ["created_date", "closed_date"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# Preserve ZIP as string (leading zeros)
if "incident_zip" in df.columns:
    df["incident_zip"] = df["incident_zip"].astype("string")

# Normalize a few key categoricals
if "borough" in df.columns:
    df["borough"] = df["borough"].astype("string").str.strip().str.upper()
    df.loc[df["borough"].isin(["", "NAN", "UNSPECIFIED"]), "borough"] = pd.NA

if "status" in df.columns:
    df["status"] = df["status"].astype("string").str.strip().str.title()

print("Normalized shape:", df.shape)
df.head()

Normalized shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30 01:51:21,NaT,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,<NA>,<NA>,nan,nan,nan,NaN,NaN
1,67746090,2026-01-30 01:51:04,NaT,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30 01:50:53,NaT,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30 01:50:52,NaT,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30 01:50:32,NaT,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


### Wrangling sanity checks  

These prevent accidentally breaking row meaning

In [5]:
assert "unique_key" in df.columns, "Expected unique_key column (row identifier)."

dup_keys = df["unique_key"].duplicated().sum()
print("Duplicate unique_key:", int(dup_keys))

missing = df.isna().mean().sort_values(ascending=False)
missing.head(15)

Duplicate unique_key: 0


closed_date         0.60766
longitude           0.01222
latitude            0.01222
incident_zip        0.00732
borough             0.00070
unique_key          0.00000
created_date        0.00000
descriptor          0.00000
complaint_type      0.00000
agency_name         0.00000
agency              0.00000
incident_address    0.00000
status              0.00000
city                0.00000
street_name         0.00000
dtype: float64

## 2B.2 - Pandas Fundamentals: Derived Columns

We will add a small set of readable derived fields:  
* `created_day`(date)
* `created_hour`, `dayofweek`, `is_weekend`
* `is_closed`
* `resolution_hours`(only when closed)

In [6]:
df2 = df.copy()

df2["created_day"] = df2["created_date"].dt.date
df2["created_hour"] = df2["created_date"].dt.hour
df2["dayofweek"] = df2["created_date"].dt.dayofweek  # 0=Mon
df2["is_weekend"] = df2["dayofweek"].isin([5, 6]).astype(int)

df2["is_closed"] = (df2["status"] == "Closed").astype(int)

df2["resolution_hours"] = (df2["closed_date"] - df2["created_date"]).dt.total_seconds() / 3600
df2.loc[df2["closed_date"].isna(), "resolution_hours"] = np.nan

# Flag suspicious durations (useful for later validation)
df2["resolution_negative"] = (df2["resolution_hours"] < 0).astype(int)
df2["resolution_over_30d"] = (df2["resolution_hours"] > (30 * 24)).astype(int)

df2[["unique_key","created_date","status","is_closed","resolution_hours","resolution_negative","resolution_over_30d"]].head(10)

,unique_key,created_date,status,is_closed,resolution_hours,resolution_negative,resolution_over_30d
0,67720523,2026-01-30 01:51:21,Unspecified,0,NaN,0,0
1,67746090,2026-01-30 01:51:04,In Progress,0,NaN,0,0
2,67758820,2026-01-30 01:50:53,In Progress,0,NaN,0,0
3,67707975,2026-01-30 01:50:52,In Progress,0,NaN,0,0
4,67771794,2026-01-30 01:50:32,In Progress,0,NaN,0,0
5,67708260,2026-01-30 01:50:14,In Progress,0,NaN,0,0
6,67771651,2026-01-30 01:48:41,In Progress,0,NaN,0,0
7,67682527,2026-01-30 01:48:20,In Progress,0,NaN,0,0
8,67720759,2026-01-30 01:45:16,In Progress,0,NaN,0,0
9,67707981,2026-01-30 01:44:53,In Progress,0,NaN,0,0


**Checkpoint - Row count should be unchanged**

In [7]:
print("Rows before:", len(df), "| after derived cols:", len(df2))

Rows before: 50000 | after derived cols: 50000


## 2C.3 - Groupby: Build Summary Tables, Then Merge Back

We build two summary tables:  
1. Daily metrics (one row per day)
2. Agency metrics (one row per agency)

Then merge them back as request-level features.  

**Guardrail:** Merges should not change row count.

In [8]:
# Daily table (unit: day)
daily = (
    df2.groupby("created_day", dropna=False)
       .agg(
           n_requests=("unique_key", "count"),
           closed_rate=("is_closed", "mean"),
           median_resolution_hours=("resolution_hours", "median"),
           p90_resolution_hours=("resolution_hours", lambda s: np.nanpercentile(s, 90) if np.isfinite(s).any() else np.nan),
       )
       .reset_index()
)

daily.head(10)

,created_day,n_requests,closed_rate,median_resolution_hours,p90_resolution_hours
0,2026-01-27,17529,0.405557,4.092778,42.981000
1,2026-01-28,16913,0.403063,2.390000,25.913028
2,2026-01-29,15278,0.317188,1.326389,6.508389
3,2026-01-30,280,0.435714,0.480972,1.098250


In [9]:
# Agency table (unit: agency)
agency = (
    df2.groupby("agency", dropna=False)
       .agg(
           agency_n_requests=("unique_key", "count"),
           agency_closed_rate=("is_closed", "mean"),
           agency_median_resolution=("resolution_hours", "median")
       )
        .reset_index()
)

agency.sort_values("agency_n_requests", ascending=False).head(10)

,agency,agency_n_requests,agency_closed_rate,agency_median_resolution
10,HPD,18618,0.239446,23.354861
8,DSNY,13160,0.060334,20.099722
11,NYPD,11651,0.981890,1.277778
6,DOT,1980,0.381313,2.400000
1,DEP,1339,0.530993,1.033333
3,DOB,766,0.191906,0.000000
5,DOHMH,624,0.200321,14.663611
2,DHS,613,0.370310,13.062222
7,DPR,531,0.190207,13.409167
14,TLC,246,0.117886,16.582778


In [10]:
# Merge summary tables
df3 = df2.merge(
    daily,
    on="created_day",
    how="left",
    validate="many_to_one"
)

df3 = df3.merge(
    agency,
    on="agency",
    how="left",
    validate="many_to_one"
)

print("Rows after merges:", len(df3))
assert len(df3) == len(df2), "Row count changed after merges, likely a many-to-many join."
print("Row count preserved")

Rows after merges: 50000
Row count preserved


## 2C.4 - Strings + Regex: Normalize Categories and Extract Structure

We will do two common tasks:  
1. Normalize `complaint_type`
2. Create keyword flags from `descriptor`
3. Extract a street number from `incident_address` (imperfect on purpose)

**Note:** Extraction should produce a quality metric (coverage/fail rate)

In [11]:
df4 = df3.copy()

# Normalize complaint_type
if "complaint_type" in df4.columns:
    df4["complaint_type_norm"] = (
        df4["complaint_type"]
          .astype("string")
          .str.strip()
          .str.lower()
    )
else:
    df4["complaint_type_norm"] = pd.NA

# Keyword flags from descriptor
if "descriptor" in df4.columns:
    desc = df4["descriptor"].astype("string").str.lower()
    df4["kw_noise"]  = desc.str.contains(r"noise|loud|music", regex=True, na=False).astype(int)
    df4["kw_rodent"] = desc.str.contains(r"rat|mice|rodent", regex=True, na=False).astype(int)
    df4["kw_water"]  = desc.str.contains(r"water|leak|flood", regex=True, na=False).astype(int)
else:
    df4["kw_noise"] = 0
    df4["kw_rodent"] = 0
    df4["kw_water"] = 0

# Regex extraction: leading digits as street number
if "incident_address" in df4.columns:
    addr = df4["incident_address"].astype("string")
    df4["street_number"] = addr.str.extract(r"^\s*(\d+)", expand=False)
    df4["street_number"] = pd.to_numeric(df4["street_number"], errors="coerce")
    fail_rate = float(df4["street_number"].isna().mean())
    print("Street number extraction fail rate:", round(fail_rate, 3))
else:
    df4["street_number"] = np.nan

df4[["complaint_type","complaint_type_norm","descriptor","kw_noise","kw_rodent","kw_water","incident_address","street_number"]].head(10)

Street number extraction fail rate: 0.081


,complaint_type,complaint_type_norm,descriptor,kw_noise,kw_rodent,kw_water,incident_address,street_number
0,Noise - Commercial,noise - commercial,Loud Music/Party,1,0,0,nan,<NA>
1,School Maintenance,school maintenance,Heating Problem,0,0,0,911 FLATBUSH AVENUE,911
2,Noise - Residential,noise - residential,Loud Music/Party,1,0,0,936 AMSTERDAM AVENUE,936
3,Noise - Residential,noise - residential,Loud Music/Party,1,0,0,190 TRANTOR PLACE,190
4,Taxi Complaint,taxi complaint,Driver Complaint - Non Passenger,0,0,0,JOHN F KENNEDY AIRPORT,<NA>
5,Snow or Ice,snow or ice,Sidewalk,0,0,0,304 CANAL STREET,304
6,Blocked Driveway,blocked driveway,Partial Access,0,0,0,3498 FORT INDEPENDENCE STREET,3498
7,Noise - Commercial,noise - commercial,Loud Music/Party,1,0,0,66-48 MYRTLE AVENUE,66
8,Snow or Ice,snow or ice,Sidewalk,0,0,0,217 NOSTRAND AVENUE,217
9,Non-Emergency Police Matter,non-emergency police matter,Trespassing,0,0,0,365 THATFORD AVENUE,365


## 2C.5 - Feature Construction: Baseline-Friendly Encodings

We will build a request-level feature table with:  
* time features
* borough
* compressed complaint categories (top-k &rarr; `other`)
* text keyword flags
* group-level features (daily + agency)

We keep `status/i_closed` and `resolution_hours` as targets/analysis fields, not necessarily safe input featuers for every modeling task.

In [12]:
df_feat = df4.copy()

# Top-k complaint categories (normalized)
k = 15
if "complaint_type_norm" in df_feat.columns:
    topk = df_feat["complaint_type_norm"].value_counts().head(k).index
    df_feat["complaint_topk"] = df_feat["complaint_type_norm"].where(df_feat["complaint_type_norm"].isin(topk), other="other")
else:
    df_feat["complaint_topk"] = "other"

# Location grouping
df_feat["borough_norm"] = df_feat["borough"].astype("string") if "borough" in df_feat.columns else pd.NA

# Missingness flag example
df_feat["zip_missing"] = df_feat["incident_zip"].isna().astype(int) if "incident_zip" in df_feat.columns else 1

keep_cols = [
    "unique_key",
    "created_date",
    "created_day",
    "created_hour",
    "dayofweek",
    "is_weekend",
    "agency",
    "borough_norm",
    "complaint_topk",
    "zip_missing",
    "kw_noise",
    "kw_rodent",
    "kw_water",
    "street_number",
    # group-level features
    "n_requests",
    "closed_rate",
    "median_resolution_hours",
    "agency_n_requests",
    "agency_closed_rate",
    "agency_median_resolution",
    # targets/analysis fields
    "status",
    "is_closed",
    "resolution_hours",
]

keep_cols = [c for c in keep_cols if c in df_feat.columns]
df_model = df_feat[keep_cols].copy()

print("Feature table shape:", df_model.shape)
df_model.head()

Feature table shape: (50000, 23)


,unique_key,created_date,created_day,created_hour,dayofweek,is_weekend,agency,borough_norm,complaint_topk,zip_missing,kw_noise,kw_rodent,kw_water,street_number,n_requests,closed_rate,median_resolution_hours,agency_n_requests,agency_closed_rate,agency_median_resolution,status,is_closed,resolution_hours
0,67720523,2026-01-30 01:51:21,2026-01-30,1,4,0,NYPD,<NA>,other,1,1,0,0,<NA>,280,0.435714,0.480972,11651,0.981890,1.277778,Unspecified,0,NaN
1,67746090,2026-01-30 01:51:04,2026-01-30,1,4,0,DOE,BROOKLYN,other,0,0,0,0,911,280,0.435714,0.480972,144,0.520833,7.605000,In Progress,0,NaN
2,67758820,2026-01-30 01:50:53,2026-01-30,1,4,0,NYPD,MANHATTAN,noise - residential,0,1,0,0,936,280,0.435714,0.480972,11651,0.981890,1.277778,In Progress,0,NaN
3,67707975,2026-01-30 01:50:52,2026-01-30,1,4,0,NYPD,STATEN ISLAND,noise - residential,0,1,0,0,190,280,0.435714,0.480972,11651,0.981890,1.277778,In Progress,0,NaN
4,67771794,2026-01-30 01:50:32,2026-01-30,1,4,0,TLC,QUEENS,other,0,0,0,0,<NA>,280,0.435714,0.480972,246,0.117886,16.582778,In Progress,0,NaN


### Minimal feature QA  

Check missing rates and uniqueness to catch surprises.

In [13]:
qa = pd.DataFrame({
    "dtype": df_model.dtypes.astype(str),
    "missing_rate": df_model.isna().mean(),
    "n_unique": df_model.nunique(dropna=True),
}).sort_values("missing_rate", ascending=False)

qa.head(20)

,dtype,missing_rate,n_unique
resolution_hours,float64,0.60766,13719
street_number,Int64,0.08100,3822
agency_median_resolution,float64,0.00374,12
borough_norm,string,0.00070,5
unique_key,int64,0.00000,50000
created_date,datetime64[ns],0.00000,39607
created_day,object,0.00000,4
agency,object,0.00000,15
is_weekend,int64,0.00000,1
dayofweek,int32,0.00000,4


## 2C.6 - Save Outputs for Later Modules

We save two artifacts:  

1. Request-level feature table (one row per request)
2. Daily feature table (one row per day)

In [14]:
WH_DIR.mkdir(parents=True, exist_ok=True)

df_model.to_parquet(OUT_REQUESTS, index=False)
daily.to_parquet(OUT_DAILY, index=False)

print("Saved request-level features:", OUT_REQUESTS)
print("Saved daily features:", OUT_DAILY)

Saved request-level features: work/m2/data/warehouse/nyc311_requests_features.parquet
Saved daily features: work/m2/data/warehouse/nyc311_daily_features.parquet


## 2C.7 - Deliverable: Transformation Log (short)

This makes your work reproducible and reviewable

In [15]:
transformation_log = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "row_definition_start": "Each row is one 311 request (unique_key).",
    "row_definition_end": "Each row is one 311 request (unique_key), with engineered features.",
    "steps": [
        "Normalized missing strings; parsed timestamps; standardized borough/status.",
        "Created time features and resolution fields (for analysis).",
        "Aggregated daily and agency metrics; merged back with validate='many_to_one'.",
        "Normalized complaint types; created descriptor keyword flags; extracted street_number via regex.",
        "Compressed complaint categories to top-k + 'other'.",
    ],
    "notes": [
        "street_number extraction is imperfect; measure fail rate and treat as noisy.",
        "resolution_hours uses closed_date and is not safe as an input feature for predicting closure outcomes (leakage risk).",
    ],
    "artifacts": {
        "request_features_parquet": str(OUT_REQUESTS),
        "daily_features_parquet": str(OUT_DAILY),
    }
}

TRANSFORM_LOG_PATH = REF_DIR / "transformation_log.json"
TRANSFORM_LOG_PATH.write_text(json.dumps(transformation_log, indent=2))
print("Saved transformation log:", TRANSFORM_LOG_PATH)

Saved transformation log: work/m2/data/reference/transformation_log.json


## 2C.8 - Quick Checkpoint

Should be able to: 
* Explain what each engineered feature means
* Verify merges did not multiple rows
* Save a tidy feature table for reuse

# Module 2.D - Scaling Data Workflows (mini pipeline)

So far we have built a small workflow:  
* **2.A:** acquire raw data &rarr; `/work/m2/data/raw/`
* **2.B:** write quality report &rarr; `/work/m2/data/reference/quality_report.json`
* **2.C:** build features tables &rarr; `/work/m2/data/warehouse/*.parquet`

This module turns that into a repeatable mini-pipeline:
1. Define **pipeline contracts** (inputs/outputs)
2. Add **pipeline tests** (fail fast)
3. Write **partitioned outputs** (by day)
4. Add **run logging** (what changed and why)
5. Optional: Compare a pandas step to **Polars**

**Note:**  
The goal is not "big data".  It is repeatability and safety

## 2D.0 - Setup: Paths + Artifacts

We will read:  
* Raw CSV: `raw/nyc311_last14d.csv`
* Report: `reference/quality_report.json`
* Features: `warehouse/nyc311_requests_features.parquet`

We will write:
* `warehouse/partitions/created_day=YYYY=MM-DD/requests.parquet`
* `reference/pipeline_runs_YYYMMDD_HHMMSS_utc.json`

In [1]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timezone
import json
import hashlib

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)

WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR = MODULE_DIR / "data"

RAW_DIR = DATA_DIR / "raw"
REF_DIR = DATA_DIR / "reference"
WH_DIR  = DATA_DIR / "warehouse"

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
REPORT_PATH = REF_DIR / "quality_report.json"
FEATURES_PATH = WH_DIR / "nyc311_requests_features.parquet"

PART_DIR = WH_DIR / "partitions"
RUN_DIR = REF_DIR / "pipeline_runs"

PART_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR.mkdir(parents=True, exist_ok=True)

print("CSV:", CSV_PATH, "| exists?", CSV_PATH.exists())
print("Report:", REPORT_PATH, "| exists?", REPORT_PATH.exists())
print("Features:", FEATURES_PATH, "| exists?", FEATURES_PATH.exists())
print("Partition root:", PART_DIR)
print("Run log dir:", RUN_DIR)

CSV: work/m2/data/raw/nyc311_last14d.csv | exists? True
Report: work/m2/data/reference/quality_report.json | exists? True
Features: work/m2/data/warehouse/nyc311_requests_features.parquet | exists? True
Partition root: work/m2/data/warehouse/partitions
Run log dir: work/m2/data/reference/pipeline_runs


## 2D.1 - Pipeline Contract: What Must Be True?

A scaled workflow starts with **contracts:**
* Required input columns
* Allowed / expected dtypes
* Row definition
* Output artifacts

If these contracts fail, the pipeline should **fail fast** instead of producing silently bad data.

In [2]:
REQUIRED_RAW_COLS = {
    "unique_key",
    "created_date",
    "agency",
    "complaint_type",
    "status",
}

OPTIONAL_RAW_COLS = {
    "closed_date",
    "borough",
    "incident_zip",
    "descriptor",
    "incident_address",
    "latitude",
    "longitude",
}

ROW_DEFINITION = "Each row is one 311 service request (unique_key)."

print("Required:", sorted(REQUIRED_RAW_COLS))
print("Optional (subset):", sorted(list(OPTIONAL_RAW_COLS))[:6], "...")
print("Row definition:", ROW_DEFINITION)


Required: ['agency', 'complaint_type', 'created_date', 'status', 'unique_key']
Optional (subset): ['borough', 'closed_date', 'descriptor', 'incident_address', 'incident_zip', 'latitude'] ...
Row definition: Each row is one 311 service request (unique_key).


**Pipeline test helpers**  

We will implement small checks that are cheap but high value

In [3]:
class PipelineError(RuntimeError):
    pass

def check_required_columns(df: pd.DataFrame, required: set[str], context: str) -> None:
    missing = sorted(list(required - set(df.columns)))
    if missing:
        raise PipelineError(f"[{context}] Missing required columns: {missing}")

def check_unique_key(df: pd.DataFrame, key: str, context: str, allow_dupes: bool = False) -> None:
    if key not in df.columns:
        raise PipelineError(f"[{context}] Missing key column: {key}")
    n_dupes = int(df[key].duplicated().sum())
    if n_dupes and not allow_dupes:
        raise PipelineError(f"[{context}] Key '{key}' has {n_dupes} duplicates (expected unique).")

def check_nonempty(df: pd.DataFrame, context: str) -> None:
    if len(df) == 0:
        raise PipelineError(f"[{context}] DataFrame is empty.")

def fingerprint_file(path: Path, n_bytes: int = 2_000_000) -> str:
    """Stable-ish fingerprint for a file (hash first N bytes + size)."""
    h = hashlib.sha256()
    with path.open("rb") as f:
        chunk = f.read(n_bytes)
    h.update(chunk)
    h.update(str(path.stat().st_size).encode("utf-8"))
    return h.hexdigest()[:16]

print("Helpers ready.")

Helpers ready.


## 2D.2 - Load Raw Data + Validate Contract 

This is where we will catch issues early:
* schema drift
* missing required fields
* broken keys

In [4]:
df_raw = pd.read_csv(CSV_PATH)
print("Raw shape:", df_raw.shape)
df_raw.head()

Raw shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [5]:
# Tests
check_nonempty(df_raw, "raw_load")
check_required_columns(df_raw, REQUIRED_RAW_COLS, "raw_load")
check_unique_key(df_raw, "unique_key", "raw_load", allow_dupes=True)  # allow in raw: handle later

raw_stats = {
    "n_rows": int(len(df_raw)),
    "n_cols": int(df_raw.shape[1]),
    "missing_rate_top": df_raw.isna().mean().sort_values(ascending=False).head(8).to_dict()
}

raw_stats

{'n_rows': 50000,
 'n_cols': 15,
 'missing_rate_top': {'closed_date': 0.60766,
  'city': 0.0764,
  'street_name': 0.02592,
  'incident_address': 0.02588,
  'latitude': 0.01222,
  'longitude': 0.01222,
  'incident_zip': 0.00732,
  'descriptor': 0.0068}}

## 2D.3 - ELT Style Staging: Normalize + Type Parse

In ELT, we often: 
* **Load** raw data first (keep a copy)
* Transform into **staged** and **curated** tables

We will create `staged` with conservative normalization similar to 2.B/2.C

In [6]:
staged = df_raw.copy()

MISSING_TOKENS = {"", " ", "  ", "UNKNOWN", "Unknown", "N/A", "NA", "NULL", "null"}

for c in staged.columns:
    if staged[c].dtype == "object":
        s = staged[c].astype(str).str.strip()
        s = s.replace({t: np.nan for t in MISSING_TOKENS})
        staged[c] = s

for c in ["created_date", "closed_date"]:
    if c in staged.columns:
        staged[c] = pd.to_datetime(staged[c], errors="coerce")

if "incident_zip" in staged.columns:
    staged["incident_zip"] = staged["incident_zip"].astype("string")

if "borough" in staged.columns:
    staged["borough"] = staged["borough"].astype("string").str.strip().str.upper()
    staged.loc[staged["borough"].isin(["", "NAN", "UNSPECIFIED"]), "borough"] = pd.NA

if "status" in staged.columns:
    staged["status"] = staged["status"].astype("string").str.strip().str.title()

print("Staged shape:", staged.shape)
staged.head()


Staged shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30 01:51:21,NaT,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,<NA>,<NA>,nan,nan,nan,NaN,NaN
1,67746090,2026-01-30 01:51:04,NaT,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30 01:50:53,NaT,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30 01:50:52,NaT,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30 01:50:32,NaT,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


**Stage tests + dedupe policy**  

We will dedupe by `unique_key` by keeping the most complete row

In [7]:
check_required_columns(staged, REQUIRED_RAW_COLS, "staged")

def dedupe_by_key_keep_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    if key not in df_in.columns:
        return df_in.copy()
    df_tmp = df_in.copy()
    df_tmp["_missing_count"] = df_tmp.isna().sum(axis=1)
    df_tmp = df_tmp.sort_values([key, "_missing_count"]).drop_duplicates(subset=[key], keep="first")
    return df_tmp.drop(columns=["_missing_count"])

before = len(staged)
staged = dedupe_by_key_keep_most_complete(staged, "unique_key")
after = len(staged)

print("Deduped rows:", before, "->", after, "| removed:", before - after)
check_unique_key(staged, "unique_key", "staged_deduped", allow_dupes=False)

Deduped rows: 50000 -> 50000 | removed: 0


## 2D.4 - Curated Table: Minimal Features + Partitioned Write

We will create a lightweight curated request table, then write Parquet aprtitions by day:  

`warehouse/partitions/created_day=YYYY-MM-DD/requests.parquet`

In [8]:
curated = staged.copy()

curated["created_day"] = curated["created_date"].dt.date
curated["created_hour"] = curated["created_date"].dt.hour
curated["dayofweek"] = curated["created_date"].dt.dayofweek
curated["is_weekend"] = curated["dayofweek"].isin([5,6]).astype(int)
curated["is_closed"] = (curated["status"] == "Closed").astype(int)

if "closed_date" in curated.columns:
    curated["resolution_hours"] = (curated["closed_date"] - curated["created_date"]).dt.total_seconds() / 3600
    curated.loc[curated["closed_date"].isna(), "resolution_hours"] = np.nan
else:
    curated["resolution_hours"] = np.nan

curated["complaint_type_norm"] = curated["complaint_type"].astype("string").str.strip().str.lower()

keep = [
    "unique_key","created_date","created_day","created_hour","dayofweek","is_weekend",
    "agency","complaint_type_norm","status","is_closed","resolution_hours",
    "borough","incident_zip","descriptor","incident_address",
]
keep = [c for c in keep if c in curated.columns]
curated = curated[keep].copy()

print("Curated shape:", curated.shape)
curated.head()

Curated shape: (50000, 15)


,unique_key,created_date,created_day,created_hour,dayofweek,is_weekend,agency,complaint_type_norm,status,is_closed,resolution_hours,borough,incident_zip,descriptor,incident_address
42208,67633298,2026-01-27 13:23:00,2026-01-27,13,1,0,DEP,hazardous materials,Open,0,NaN,BROOKLYN,11211.0,Chemical Spill/Release (HA1),774 DRIGGS AVENUE
42322,67633300,2026-01-27 13:17:00,2026-01-27,13,1,0,DEP,water system,Started,0,NaN,BROOKLYN,11207.0,Excessive Water In Basement (WEFB),593 WARWICK STREET
48647,67633301,2026-01-27 08:26:00,2026-01-27,8,1,0,DEP,air quality,Closed,1,49.566667,BROOKLYN,11206.0,"Air: Odor/Fumes, Vehicle Idling (AD3)",nan
39232,67633304,2026-01-27 16:05:00,2026-01-27,16,1,0,DEP,air quality,Started,0,NaN,BROOKLYN,11249.0,"Air: Odor/Fumes, Vehicle Idling (AD3)",52 RIVER STREET
44498,67633305,2026-01-27 11:26:00,2026-01-27,11,1,0,DEP,asbestos,Open,0,NaN,QUEENS,11104.0,Asbestos Complaint (B1),45-55 40 STREET


**Partition Write**  

One Parquet file per day


In [9]:
n_written = 0
example_partition = None

for day, chunk in curated.groupby("created_day", dropna=False):
    day_str = str(day)
    out_dir = PART_DIR / f"created_day={day_str}"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / "requests.parquet"
    chunk.to_parquet(out_path, index=False)
    n_written += 1
    if example_partition is None:
        example_partition = out_path

print("Wrote partitions:", n_written)
print("Example:", example_partition)

Wrote partitions: 4
Example: work/m2/data/warehouse/partitions/created_day=2026-01-27/requests.parquet


**Read back sanity check**  


In [10]:
some_parts = sorted(PART_DIR.glob("created_day=*/requests.parquet"))[:3]
print("Checking partitions:", len(some_parts))

for p in some_parts:
    x = pd.read_parquet(p)
    check_nonempty(x, f"readback:{p.parent.name}")
    check_required_columns(x, {"unique_key","created_date","agency","complaint_type_norm","status"}, f"readback:{p.parent.name}")
    check_unique_key(x, "unique_key", f"readback:{p.parent.name}", allow_dupes=False)
    print("OK:", p.parent.name, "rows:", len(x))

Checking partitions: 3
OK: created_day=2026-01-27 rows: 17529
OK: created_day=2026-01-28 rows: 16913
OK: created_day=2026-01-29 rows: 15278


## 2D.5 - Pipeline Run Log

We log:  

* input fingerprint
* row counts before/after dedupe
* partitions written
* example output path

In [11]:
input_fp = fingerprint_file(CSV_PATH) if CSV_PATH.exists() else None

run = {
    "run_id": datetime.now(timezone.utc).strftime("run_%Y%m%d_%H%M%S_utc"),
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "row_definition": ROW_DEFINITION,
    "inputs": {
        "csv_path": str(CSV_PATH),
        "csv_fingerprint": input_fp,
    },
    "checks": {
        "required_raw_cols": sorted(REQUIRED_RAW_COLS),
        "dedupe_policy": "keep most complete row per unique_key",
    },
    "counts": {
        "raw_rows": int(len(df_raw)),
        "staged_rows_after_dedupe": int(len(staged)),
        "curated_rows": int(len(curated)),
        "partitions_written": int(n_written),
    },
    "outputs": {
        "partition_root": str(PART_DIR),
        "example_partition": str(example_partition),
    },
}

RUN_PATH = RUN_DIR / f"{run['run_id']}.json"
RUN_PATH.write_text(json.dumps(run, indent=2))
print("Saved run log:", RUN_PATH)
run


Saved run log: work/m2/data/reference/pipeline_runs/run_20260202_211848_utc.json


{'run_id': 'run_20260202_211848_utc',
 'generated_at_utc': '2026-02-02T21:18:48.749545+00:00',
 'row_definition': 'Each row is one 311 service request (unique_key).',
 'inputs': {'csv_path': 'work/m2/data/raw/nyc311_last14d.csv',
  'csv_fingerprint': 'e378ad4da4be2200'},
 'checks': {'required_raw_cols': ['agency',
   'complaint_type',
   'created_date',
   'status',
   'unique_key'],
  'dedupe_policy': 'keep most complete row per unique_key'},
 'counts': {'raw_rows': 50000,
  'staged_rows_after_dedupe': 50000,
  'curated_rows': 50000,
  'partitions_written': 4},
 'outputs': {'partition_root': 'work/m2/data/warehouse/partitions',
  'example_partition': 'work/m2/data/warehouse/partitions/created_day=2026-01-27/requests.parquet'}}

## 2D.6 - ELT vs ETL: Interpret What Was Built

We just implemented an ELT-style workflow:  
* Extract + load raw CSV
* Transform into staged / curated tables
* Write curated, partitioned artifacts

In notes explain:  
* Why this is ELT and not ETL
* What is the "system of record"
* What contracts were enforced


In [12]:
elt_notes = []

elt_notes.extend([
    # "This is ELT because we first load raw data as-is, then transform into staged/curated outputs.",
    # "System of record: the raw CSV (raw folder) + run logs for reproducibility.",
    # "Contracts: required columns, unique_key uniqueness after staging, partition read-back checks.",
])

elt_notes

[]

## 2D.7 - Polars Comparison

In [14]:
import polars as pl
from pathlib import Path

# Read a small number of partitions to keep this fast
PART_DIR = Path("work") / "m2" / "data" / "warehouse" / "partitions"
paths = [str(p) for p in sorted(PART_DIR.glob("created_day=*/requests.parquet"))[:10]]

print("Reading partitions:", len(paths))

# Lazy scan of Parquet partitions
lf = pl.scan_parquet(paths)

# Groupby aggregation (lazy → optimized → executed)
agency_summary = (
    lf.group_by("agency")
      .agg([
          pl.len().alias("n_requests"),          
          pl.mean("is_closed").alias("closed_rate"),
      ])
      .sort("n_requests", descending=True)
      .collect()
)

agency_summary.head(10)

Reading partitions: 4


agency,n_requests,closed_rate
str,u32,f64
"""HPD""",18618,0.239446
"""DSNY""",13160,0.060334
"""NYPD""",11651,0.98189
"""DOT""",1980,0.381313
"""DEP""",1339,0.530993
"""DOB""",766,0.191906
"""DOHMH""",624,0.200321
"""DHS""",613,0.37031
"""DPR""",531,0.190207


## 2D.8 - Quick Checkpoint

We should now have the core pieces of a scaled workflow:  
* input contracts + fail-fast checks
* staged + curated layers
* partitioned Parquet outputs
* run logging for reproducibility

# Module 2.E - Outliers & Validation (NYC 311)

This notebook builds on **2.A - 2.D**  

You will:

1. Load the request-level feature table created in **2.C** (or rebuild from 2.D partitions if needed)
2. Detect anomalies (outliers that violate expectations)
3. Write validation checks (contracts/tests you can run every pipeline execution)
4. Do a leakage audit: identify fields/transforms that can leak future information
5. Export a reusable `validation_report.json`

**Principle:** Prefer flags and tests over deleting data. Make assumptions explicit and enforcable.

## 2E.0 - Setup and Paths

Expected artifacts from prior sections:  
* Raw CSV (**2.A**): `/work/m2/data/raw/nyc311_last14d.csv`
* Quality report (**2.B**): `/work/m2/data/reference/quality_report.json`
* Feature table (**2.C**): `/work/m2/data/warehouse/nyc311_requests_features.parquet`
* Partitioned curated outputs (**2.D**): `/work/m2/data/warehouse/partitions/created_day=YYYY-MM-DD/requests.parquet`

We will write:  
* `/work/m2/data/reference/validation_report.json`

In [15]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timezone
import json

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 140)
pd.set_option("display.width", 160)

WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR = MODULE_DIR / "data"

RAW_DIR = DATA_DIR / "raw"
REF_DIR = DATA_DIR / "reference"
WH_DIR  = DATA_DIR / "warehouse"

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
QUALITY_PATH = REF_DIR / "quality_report.json"
FEATURES_PATH = WH_DIR / "nyc311_requests_features.parquet"
PART_DIR = WH_DIR / "partitions"

VALIDATION_PATH = REF_DIR / "validation_report.json"

print("CSV:", CSV_PATH, "| exists?", CSV_PATH.exists())
print("Quality report:", QUALITY_PATH, "| exists?", QUALITY_PATH.exists())
print("Features:", FEATURES_PATH, "| exists?", FEATURES_PATH.exists())
print("Partitions:", PART_DIR, "| exists?", PART_DIR.exists())
print("Will write:", VALIDATION_PATH)

CSV: work/m2/data/raw/nyc311_last14d.csv | exists? True
Quality report: work/m2/data/reference/quality_report.json | exists? True
Features: work/m2/data/warehouse/nyc311_requests_features.parquet | exists? True
Partitions: work/m2/data/warehouse/partitions | exists? True
Will write: work/m2/data/reference/validation_report.json


## 2E.1 - Load Request-Level Features (Preferred)

If `nyc311_requests_features.parquet` exists (from 2.C), we use it. Otherwise, we fall back to the curated partitions from 2.D and concatenate them.  

This module is robust to either artifact.

In [16]:
def load_features() -> pd.DataFrame:
    if FEATURES_PATH.exists():
        df = pd.read_parquet(FEATURES_PATH)
        df["_source"] = "2.C_features_parquet"
        return df

    if PART_DIR.exists():
        parts = sorted(PART_DIR.glob("created_day=*/requests.parquet"))
        if not parts:
            raise FileNotFoundError("No partitions found in warehouse/partitions/")
        df = pd.concat([pd.read_parquet(p) for p in parts], ignore_index=True)
        df["_source"] = "2.D_partitioned_curated"
        return df

    raise FileNotFoundError("No features parquet and no partitions found. Run 2.C or 2.D first.")

df = load_features()
print("Loaded:", df["_source"].iloc[0])
print("Shape:", df.shape)
df.head()

Loaded: 2.C_features_parquet
Shape: (50000, 24)


,unique_key,created_date,created_day,created_hour,dayofweek,is_weekend,agency,borough_norm,complaint_topk,zip_missing,kw_noise,kw_rodent,kw_water,street_number,n_requests,closed_rate,median_resolution_hours,agency_n_requests,agency_closed_rate,agency_median_resolution,status,is_closed,resolution_hours,_source
0,67720523,2026-01-30 01:51:21,2026-01-30,1,4,0,NYPD,<NA>,other,1,1,0,0,<NA>,280,0.435714,0.480972,11651,0.981890,1.277778,Unspecified,0,NaN,2.C_features_parquet
1,67746090,2026-01-30 01:51:04,2026-01-30,1,4,0,DOE,BROOKLYN,other,0,0,0,0,911,280,0.435714,0.480972,144,0.520833,7.605000,In Progress,0,NaN,2.C_features_parquet
2,67758820,2026-01-30 01:50:53,2026-01-30,1,4,0,NYPD,MANHATTAN,noise - residential,0,1,0,0,936,280,0.435714,0.480972,11651,0.981890,1.277778,In Progress,0,NaN,2.C_features_parquet
3,67707975,2026-01-30 01:50:52,2026-01-30,1,4,0,NYPD,STATEN ISLAND,noise - residential,0,1,0,0,190,280,0.435714,0.480972,11651,0.981890,1.277778,In Progress,0,NaN,2.C_features_parquet
4,67771794,2026-01-30 01:50:32,2026-01-30,1,4,0,TLC,QUEENS,other,0,0,0,0,<NA>,280,0.435714,0.480972,246,0.117886,16.582778,In Progress,0,NaN,2.C_features_parquet


**Normalize a few types**  

Artifacts differ slightly across 2.C vs 2.D. Normalize the most important fields:
* timestamps
* `created_day`
* `status`
* `is_closed`
* `resolution_hours`


In [17]:
df2 = df.copy()

if "created_date" in df2.columns and not np.issubdtype(df2["created_date"].dtype, np.datetime64):
    df2["created_date"] = pd.to_datetime(df2["created_date"], errors="coerce")

if "created_day" in df2.columns:
    if df2["created_day"].dtype == "object":
        df2["created_day"] = pd.to_datetime(df2["created_day"], errors="coerce").dt.date
else:
    if "created_date" in df2.columns:
        df2["created_day"] = df2["created_date"].dt.date

if "status" in df2.columns:
    df2["status"] = df2["status"].astype("string").str.strip().str.title()

if "is_closed" not in df2.columns and "status" in df2.columns:
    df2["is_closed"] = (df2["status"] == "Closed").astype(int)

if "resolution_hours" in df2.columns:
    df2["resolution_hours"] = pd.to_numeric(df2["resolution_hours"], errors="coerce")

print("Ready. Example columns present:", [c for c in ["unique_key","created_date","created_day","agency","status","is_closed","resolution_hours"] if c in df2.columns])
df2.head(3)

Ready. Example columns present: ['unique_key', 'created_date', 'created_day', 'agency', 'status', 'is_closed', 'resolution_hours']


,unique_key,created_date,created_day,created_hour,dayofweek,is_weekend,agency,borough_norm,complaint_topk,zip_missing,kw_noise,kw_rodent,kw_water,street_number,n_requests,closed_rate,median_resolution_hours,agency_n_requests,agency_closed_rate,agency_median_resolution,status,is_closed,resolution_hours,_source
0,67720523,2026-01-30 01:51:21,2026-01-30,1,4,0,NYPD,<NA>,other,1,1,0,0,<NA>,280,0.435714,0.480972,11651,0.981890,1.277778,Unspecified,0,NaN,2.C_features_parquet
1,67746090,2026-01-30 01:51:04,2026-01-30,1,4,0,DOE,BROOKLYN,other,0,0,0,0,911,280,0.435714,0.480972,144,0.520833,7.605000,In Progress,0,NaN,2.C_features_parquet
2,67758820,2026-01-30 01:50:53,2026-01-30,1,4,0,NYPD,MANHATTAN,noise - residential,0,1,0,0,936,280,0.435714,0.480972,11651,0.981890,1.277778,In Progress,0,NaN,2.C_features_parquet


## 2E.2 - Anomaly Detection: Build Flags

Outliers are not "bad rows" by default  

We flag anomalies using explicit expectations:  
* duplicate keys (should be unique at request level)
* missing future timestamps
* negative or extremely long durations
* closed but missing duration

In [18]:
flags = pd.DataFrame(index=df2.index)

# Duplicate key flag
if "unique_key" in df2.columns:
    flags["anom_duplicate_unique_key"] = df2["unique_key"].duplicated().astype(int)
else:
    flags["anom_duplicate_unique_key"] = 0

# Timestamp flags
# Robust "future" check that handles tz-naive vs tz-aware timestamps
if "created_date" in df2.columns:
    cd = pd.to_datetime(df2["created_date"], errors="coerce")

    flags["anom_created_missing"] = cd.isna().astype(int)

    # Compare to "now" with matching tz-awareness
    if getattr(cd.dt, "tz", None) is None:
        now = pd.Timestamp.now()  # tz-naive
    else:
        now = pd.Timestamp.now(tz=cd.dt.tz)  # tz-aware, matching cd

    flags["anom_created_in_future"] = (cd > now).astype(int)
else:
    flags["anom_created_missing"] = 1
    flags["anom_created_in_future"] = 0

# Duration flags (if present)
if "resolution_hours" in df2.columns:
    flags["anom_negative_duration"] = (df2["resolution_hours"] < 0).astype(int)
    flags["anom_over_30d"] = (df2["resolution_hours"] > 30*24).astype(int)
    flags["anom_over_7d"] = (df2["resolution_hours"] > 7*24).astype(int)
else:
    flags["anom_negative_duration"] = 0
    flags["anom_over_30d"] = 0
    flags["anom_over_7d"] = 0

# Consistency: closed but missing duration (when duration exists)
if "is_closed" in df2.columns and "resolution_hours" in df2.columns:
    flags["anom_closed_missing_duration"] = ((df2["is_closed"] == 1) & (df2["resolution_hours"].isna())).astype(int)
else:
    flags["anom_closed_missing_duration"] = 0

flags["anom_any"] = (flags.sum(axis=1) > 0).astype(int)

anom_rate = flags.mean().sort_values(ascending=False)
anom_rate.head(12)

anom_negative_duration          0.00004
anom_any                        0.00004
anom_created_missing            0.00000
anom_duplicate_unique_key       0.00000
anom_created_in_future          0.00000
anom_over_30d                   0.00000
anom_over_7d                    0.00000
anom_closed_missing_duration    0.00000
dtype: float64

**Investigation table (what to inspect next)**

In [19]:
cols_for_view = [c for c in ["unique_key","created_date","created_day","agency","status","is_closed","resolution_hours"] if c in df2.columns]
anom_cols = [c for c in flags.columns if c.startswith("anom_") and c != "anom_any"]

investigation = pd.concat([df2[cols_for_view], flags[["anom_any"] + anom_cols]], axis=1)
suspects = investigation.loc[investigation["anom_any"] == 1].copy()

print("Suspicious rows:", len(suspects), "out of", len(df2), f"({len(suspects)/max(1,len(df2)):.2%})")
suspects.head(25)

Suspicious rows: 2 out of 50000 (0.00%)


,unique_key,created_date,created_day,agency,status,is_closed,resolution_hours,anom_any,anom_duplicate_unique_key,anom_created_missing,anom_created_in_future,anom_negative_duration,anom_over_30d,anom_over_7d,anom_closed_missing_duration
27005,67655113,2026-01-28 10:20:00,2026-01-28,DOT,Pending,0,-24.016667,1,0,0,0,1,0,0,0
46061,67648428,2026-01-27 10:12:00,2026-01-27,DOT,Pending,0,-96.000000,1,0,0,0,1,0,0,0


## 2E.3 - Distribution Checks: Cheap Drift Signals

We compute a few "canary" summaries that can highlight drift:  
* requests per day
* top agencies
* top complaint categories (top-k or normalized)

In [20]:
summaries = {}

if "created_day" in df2.columns:
    by_day = df2.groupby("created_day").size().rename("n_requests").reset_index()
    summaries["by_day"] = by_day
    display(by_day.tail(10))

if "agency" in df2.columns:
    by_agency = df2["agency"].astype("string").value_counts(dropna=False).head(15).rename_axis("agency").reset_index(name="n")
    summaries["by_agency_top"] = by_agency
    display(by_agency)

cat_col = None
for candidate in ["complaint_topk", "complaint_type_norm", "complaint_type"]:
    if candidate in df2.columns:
        cat_col = candidate
        break

if cat_col:
    by_cat = df2[cat_col].astype("string").value_counts(dropna=False).head(15).rename_axis(cat_col).reset_index(name="n")
    summaries["by_category_top"] = by_cat
    display(by_cat)
else:
    print("No complaint category column found.")


,created_day,n_requests
0,2026-01-27,17529
1,2026-01-28,16913
2,2026-01-29,15278
3,2026-01-30,280


,agency,n
0,HPD,18618
1,DSNY,13160
2,NYPD,11651
3,DOT,1980
4,DEP,1339
5,DOB,766
6,DOHMH,624
7,DHS,613
8,DPR,531
9,TLC,246


,complaint_topk,n
0,snow or ice,12376
1,heat/hot water,11890
2,other,7902
3,illegal parking,4922
4,blocked driveway,2790
5,noise - residential,2492
6,unsanitary condition,1288
7,plumbing,1258
8,paint/plaster,819
9,door/window,773


**Simple spike detection (rule-based)**  

Flag days where request count is unusually high relative to median

In [21]:
spike_report = None

if "created_day" in df2.columns:
    by_day = df2.groupby("created_day").size().rename("n_requests").reset_index()
    med = float(by_day["n_requests"].median()) if len(by_day) else 0.0
    by_day["is_spike"] = (by_day["n_requests"] > 2.5 * med).astype(int)
    by_day["median_day_requests"] = med
    spike_report = by_day.loc[by_day["is_spike"] == 1].copy()

    print("Median day requests:", med)
    print("Spike days:", len(spike_report))
    display(spike_report)
else:
    print("No created_day available.")

Median day requests: 16095.5
Spike days: 0


,created_day,n_requests,is_spike,median_day_requests


## 2E.4 - Validation Checks: Fail-Fast Contracts (Report-First)

A scalable pipeline should run a small set of checks on every run. We create a report that can either:  
* be used in dev (raise on failure), or
* be used in production (alert/stop/quarantine)

Checks:
* required columns
* uniqueness
* timestamp validity
* duration plausibility (if present)
* missingness guardrails for key fields

In [22]:
def run_validations(df_in: pd.DataFrame) -> dict:
    results = {"passed": True, "checks": []}

    def record(name: str, passed: bool, details: dict):
        results["checks"].append(
            {"name": name, "passed": bool(passed), "details": details}
        )
        if not passed:
            results["passed"] = False

    # --------------------------------------------------
    # Required columns
    # --------------------------------------------------
    required = ["unique_key", "created_date", "agency"]
    missing_required = [c for c in required if c not in df_in.columns]
    record(
        "required_columns",
        passed=(len(missing_required) == 0),
        details={"required": required, "missing": missing_required},
    )

    # --------------------------------------------------
    # Uniqueness
    # --------------------------------------------------
    if "unique_key" in df_in.columns:
        dupes = int(df_in["unique_key"].duplicated().sum())
        record(
            "unique_key_unique",
            passed=(dupes == 0),
            details={"duplicate_count": dupes},
        )
    else:
        record(
            "unique_key_unique",
            passed=False,
            details={"error": "unique_key not present"},
        )

    # --------------------------------------------------
    # created_date checks
    # --------------------------------------------------
    if "created_date" in df_in.columns:
        cd = pd.to_datetime(df_in["created_date"], errors="coerce")

        missing_rate = float(cd.isna().mean())
        record(
            "created_date_missing_rate",
            passed=(missing_rate < 0.01),
            details={"missing_rate": missing_rate, "threshold": 0.01},
        )

        # Match timezone awareness
        if getattr(cd.dt, "tz", None) is None:
            now = pd.Timestamp.now()
        else:
            now = pd.Timestamp.now(tz=cd.dt.tz)

        future = int((cd > now).sum())
        record(
            "created_date_future_count",
            passed=(future == 0),
            details={"future_count": future},
        )
    else:
        record(
            "created_date_missing_rate",
            passed=False,
            details={"error": "created_date not present"},
        )
        record(
            "created_date_future_count",
            passed=False,
            details={"error": "created_date not present"},
        )

    # --------------------------------------------------
    # Duration plausibility (optional)
    # --------------------------------------------------
    if "resolution_hours" in df_in.columns:
        rh = pd.to_numeric(df_in["resolution_hours"], errors="coerce")

        neg = int((rh < 0).sum())
        record(
            "resolution_negative_count",
            passed=(neg == 0),
            details={"negative_count": neg},
        )

        over_30d = int((rh > 30 * 24).sum())
        record(
            "resolution_over_30d_count",
            passed=(over_30d == 0),
            details={"over_30d_count": over_30d},
        )
    else:
        record(
            "resolution_checks_skipped",
            passed=True,
            details={"note": "resolution_hours not present"},
        )

    # --------------------------------------------------
    # Missingness guardrails (high-value fields)
    # --------------------------------------------------
    for col, thresh in [
        ("agency", 0.02),
        ("status", 0.02),
        ("complaint_topk", 0.10),
        ("complaint_type_norm", 0.10),
    ]:
        if col in df_in.columns:
            miss = float(df_in[col].isna().mean())
            record(
                f"missing_rate:{col}",
                passed=(miss <= thresh),
                details={"missing_rate": miss, "threshold": thresh},
            )

    return results


validation = run_validations(df2)
print("Validation passed?", validation["passed"])


Validation passed? False


In [23]:
fails = [c for c in validation["checks"] if not c["passed"]]
print("Failures:", len(fails))
for f in fails:
    print("-", f["name"], f["details"])


Failures: 1
- resolution_negative_count {'negative_count': 2}


## 2E.5 - Leakage Audit

Leakage is any information your model would not have at prediction time.  

We generate:  
* a list of risky columns present
* a checklist you can use during feature engineering

In [24]:
LEAKAGE_RISK_COLS = {
    "closed_date": "Outcome timestamp: only known after resolution.",
    "resolution_hours": "Derived from closed_date; not safe as an input feature for closure-related prediction tasks.",
    "status": "Often a label/outcome; may leak if used as an input feature for certain tasks.",
}

present_risks = {c: LEAKAGE_RISK_COLS[c] for c in LEAKAGE_RISK_COLS if c in df2.columns}

leakage_audit = {
    "present_risky_columns": present_risks,
    "checklist": [
        "Any preprocessing stats (means/encoders/top-k) computed on TRAIN only?",
        "Any joins pulling in future updates (time-insensitive joins)?",
        "Any imputation using target-dependent information?",
        "Any outcome-derived fields included as features?",
        "Is row meaning tied to a timestamp (prediction time)?",
    ],
    "notes": [
        "If modeling 'is_closed' or 'resolution_hours', treat resolution_hours (and closed timestamps) as labels/targets, not inputs.",
        "In later modules, fit encoders/imputers on TRAIN only (Pipeline/ColumnTransformer).",
    ],
}

print("Risky columns present:", list(present_risks.keys()))
leakage_audit

Risky columns present: ['resolution_hours', 'status']


{'present_risky_columns': {'resolution_hours': 'Derived from closed_date; not safe as an input feature for closure-related prediction tasks.',
  'status': 'Often a label/outcome; may leak if used as an input feature for certain tasks.'},
 'checklist': ['Any preprocessing stats (means/encoders/top-k) computed on TRAIN only?',
  'Any joins pulling in future updates (time-insensitive joins)?',
  'Any imputation using target-dependent information?',
  'Any outcome-derived fields included as features?',
  'Is row meaning tied to a timestamp (prediction time)?'],
 'notes': ["If modeling 'is_closed' or 'resolution_hours', treat resolution_hours (and closed timestamps) as labels/targets, not inputs.",
  'In later modules, fit encoders/imputers on TRAIN only (Pipeline/ColumnTransformer).']}

## 2E.6 - Write `validation_report.json`

This is the artifact that can saved on every pipeline run and compare over time

In [25]:
report = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "data_source": str(df["_source"].iloc[0]) if "_source" in df.columns else "unknown",
    "n_rows": int(len(df2)),
    "n_cols": int(df2.shape[1]),
    "anomaly_rates": flags.mean().sort_values(ascending=False).to_dict(),
    "n_suspicious_rows": int(flags["anom_any"].sum()),
    "validation": validation,
    "leakage_audit": leakage_audit,
    "distribution_canaries": {},
}

if "created_day" in df2.columns:
    by_day = df2.groupby("created_day").size()
    report["distribution_canaries"]["day_count_summary"] = {
        "n_days": int(by_day.shape[0]),
        "min": int(by_day.min()) if len(by_day) else 0,
        "median": float(by_day.median()) if len(by_day) else 0.0,
        "max": int(by_day.max()) if len(by_day) else 0,
    }

if "agency" in df2.columns:
    top_ag = df2["agency"].astype("string").value_counts(dropna=False).head(10)
    report["distribution_canaries"]["top_agencies"] = top_ag.to_dict()

if cat_col:
    top_cat = df2[cat_col].astype("string").value_counts(dropna=False).head(10)
    report["distribution_canaries"][f"top_categories:{cat_col}"] = top_cat.to_dict()

if spike_report is not None:
    report["distribution_canaries"]["spike_days"] = spike_report.to_dict(orient="records")

REF_DIR.mkdir(parents=True, exist_ok=True)
VALIDATION_PATH.write_text(json.dumps(report, indent=2, default=str))
print("Saved:", VALIDATION_PATH)

Saved: work/m2/data/reference/validation_report.json


## 2E.7 - Deliverable: Pick Policies for Anomalies

For each anomaly type, decide a policy:
* **Fix**: correct obvious errors
* **Drop:** rarely ideal; loses information
* **Flag:** recommended default
* **Quarantine:** exclude from modeling but keep for investigation

Fill in and customize

In [26]:
anomaly_policies = {
    "anom_duplicate_unique_key": "flag + investigate upstream, consider dedupe in staging",
    "anom_created_in_future": "quarantine + investigate timezone/parsing bugs",
    "anom_negative_duration": "flag + investigate; consider setting duration to NaN",
    "anom_over_30d": "flag; may be real; consider robust models or caps depending on goal",
    "anom_closed_missing_duration": "flag; treat duration unknown; check status/timestamp consistency",
}

anomaly_policies

{'anom_duplicate_unique_key': 'flag + investigate upstream, consider dedupe in staging',
 'anom_created_in_future': 'quarantine + investigate timezone/parsing bugs',
 'anom_negative_duration': 'flag + investigate; consider setting duration to NaN',
 'anom_over_30d': 'flag; may be real; consider robust models or caps depending on goal',
 'anom_closed_missing_duration': 'flag; treat duration unknown; check status/timestamp consistency'}

## 2E.8 - Quick Checkpoint

We should now have:
* Anomaly flags + an investigation table
* Fail-fast validations (contracts)
* A leakage audit checklist
* A reusable `validation_report.json` artifact

# F